### Functions
(!) Pozor je nutné zkontrolovat, že tam jsou všechny namespacy

In [13]:
from lxml import etree as ElementTree
import pandas as pd

# Nahrazení namespaců, aby bylo lépe čitelné

or_namespaces = {'xsd': 'http://www.w3.org/2001/XMLSchema'
             }
inverse_or_namespaces = {v: k for k, v in or_namespaces.items()}
def replace_ns(path_w_ns):
    for key, name in inverse_or_namespaces.items():
        path_w_ns = path_w_ns.replace(key, name)
    path_wo_ns = path_w_ns
    path_wo_ns = path_wo_ns.replace('{','')
    path_wo_ns = path_wo_ns.replace('}',':')
    return path_wo_ns

# Elementy XML a jejich xpath cesty
def etree_iter_path(node, tag=None, path='.'):
    if tag == "*":
        tag = None
    if tag is None or node.tag == tag:
        yield node, path
    for child in node:
        _child_path = '%s/%s' % (path, child.tag)
        for child, child_path in etree_iter_path(child, tag, path=_child_path):
            yield child, child_path

# Dotažení info o XML do pd.DataFrame
def xml_paths_no_dupl(filename):
    xmldoc = ElementTree.parse(filename)
    list_data = []
    for elem, path in etree_iter_path(xmldoc.getroot()):
        elem_parent = ''
        if elem.getparent() is not None:
            elem_parent = replace_ns(elem.getparent().tag)
        attribu = elem.attrib
        attr_keys_string = ",".join(attribu.keys())
        list_data.append([replace_ns(elem.tag), elem_parent,attr_keys_string,replace_ns(path)])
    pd_data = pd.DataFrame(list_data, columns = ['Tag','Parent_Tag','Attributes','xPath'])
    pd_no_dupl = pd_data.drop_duplicates()
    return pd_no_dupl

### Profile xml files
1. Check xml files folder
1. Check all xpaths and return dataframe
1. Export data frame to excel

In [14]:
import os
files_to_profile = os.listdir('./xsd')
files_to_profile

['ares_datatypes_v_1.0.2.xsd']

In [15]:
complet = pd.DataFrame(columns = ['Tag','Parent_Tag','Attributes','xPath'])

for file in files_to_profile:
    filename = './xsd/'+file
    result = xml_paths_no_dupl(filename)
    complet = pd.concat([complet, result])
comp_no_dup=complet.drop_duplicates()

In [17]:
"""Zapsání """
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('xsd_profile.xlsx')

# Convert the dataframe to an XlsxWriter Excel object.
comp_no_dup.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()